<div style="text-align: center;">

<h1 style= "color:yellowgreen">Image representation and OpenCV in ROS
</h1>

</div>


<h2 style= "color:coral"> Images Represented in Code
</h2>

#### Gray scale images
Images are composed by pixels.   
Each pixel is represented by an **intensity value**.
In grayscale, each value varies between 0 (black color) to 255 (white color).   
The values in between will give you different intensities of gray as shown in the figure below.

<br><img src="../assets/ex4-imgs/grayscale.png" style="width: 50%; height: auto;"></br>

An image is the a matrix of pixels as shown in the **16 x 12** image below.   
In other words, images are represented by a matrix of intensity values.

<br><img src="../assets/ex4-imgs/grayscale-image.png" style="width: 75%; height: auto;"></br>
<h6><p style="text-align: center;">
<strong>Image from </strong><a href="https://openframeworks.cc/ofBook/chapters/image_processing_computer_vision.html">Image Processing</a>
</p></h6>


<h3> <span style= "color:red">What </span><span style= "color:green"> about </span><span style= "color:blue"> colors? </span></h3>


#### RGB images

Color is introduced is model similar manner.  
Three matrices with intensity values for colors <span style= "color:red">Red </span><span style= "color:green"> Green </span><span style= "color:blue"> Blue </span> (RGB)
<br><img src="../assets/ex4-imgs/rgb-array.png" style="width: 50%; height: auto;"></br>
<strong>Image from </strong><a href="https://www.geeksforgeeks.org/matlab-rgb-image-representation/">RGB Image</a>




<h2 style= "color:coral"> OpenCV Image Representation and Basic Operations
</h2>

- Library for image processing and computer vision algorithms
- Images stored as multi-dim NumPy arrays
- Default color images are in BGR format 🤷🏾‍♂️


### Images in ROS and OpenCV

- ROS has its own Image message type (different from OpenCV) - `sensor_msgs/Image`
<br><img src="../assets/ex4-imgs/ROS-image-message.png" style="width: 50%; height: auto;"></br>

- Package cv_bridge converts ROS Images to OpenCV images and vice-versa
- The CvBridge class provides two key functions:
	- `imgmsg_to_cv2`: Converts a ROS image (sensor_msgs/Image) to an OpenCV image.
	- `cv2_to_imgmsg`: Converts an OpenCV image to a ROS image for publishing.
	- In both function you need an encoding type 
		- mono8 - grayscale
		- bgr8 - blue, green, red encoding

<strong><a href="http://wiki.ros.org/cv_bridge/Tutorials/ConvertingBetweenROSImagesAndOpenCVImagesPython">More info on CVBridge </a></strong>


In [ ]:
#THIS CODE IS JUST TO SHOW THE CV BRIDGE FUNCTIONS
bridge = CvBridge() #instantiate the bridge class
cv_img = cv2.imread(filename) #For completeness: reads an image
img_msg = bridge.cv2_to_imgmsg(cv_img, "bgr8") #Converts an cv image to an ROS Image message - "bgr8" is the encoding
cv_img = bridge.imgmsg_to_cv2(img_msg, "bgr8") #Converts ROS Image message to an cv Image - "bgr8" is the encoding

### OpenCV - NumPy operations

- Some useful cv2 commands

In [ ]:
img =  cv2.imread("test_cropped.jpg") #HERE JUST FOR COMPLETENESS - we won't use read (Images will come from topics for us)
image_copy = img.copy() #Copy images
imgheight=img.shape[0] #Gets number of rows (height)
imgwidth=img.shape[1] #Gets number of columns (width)

- Via NumPy indexing

In [ ]:
pixel = image[i, j, k] # Access pixel at row i, column j, and channel k (0 = Blue, 1= Green, 2=Red)

- Cropping using array slicing

In [ ]:
cropped_image = image[start_row:end_row, start_col:end_col] #crops image rows (height) from start to end and columns (width) start to end
cropped_image = image[start_row:end_row] #crops only image rows (height) from start to end
cropped_image = image[:,start_col:end_col] #crops only image columns (width) from start to end

**Test the `crop.py` Node in the `img_proc_aux` package**

In the **VNC container**, (`dts code workbench --recipe ../ex4-recipe`)

- Open three LXTerminals
- Build and source the catkin workspace
	- roscore 
	- rosrun img_proc_aux image_pub_all.py
	- rosrun img_proc_aux crop.py
- To view the image
	- In the VNC container, open the rqt_image_view icon.

In the **Devel container**, (`dts devel run -X -M -f --cmd bash`)

- Open terminals as necessary (with `dts devel run attach`)
- Build and source the catkin workspace
	- roscore (one terminal)
	- rosrun img_proc_aux image_pub_all.py (one terminal)
	- rosrun img_proc_aux crop.py (one terminal)
- To view the image
	- In the Devel container, attach a new terminal  and run the command `rqt_image_view` (one terminal)

In [ ]:
#!/usr/bin/env python3

import sys
import rospy
import cv2
from sensor_msgs.msg import Image
from cv_bridge import CvBridge

class ImageCropper:
    def __init__(self):
        # Instatiate the converter class once by using a class member
        self.bridge = CvBridge()
        rospy.Subscriber("image", Image, self.crop)
        self.pub = rospy.Publisher("image_cropped", Image, queue_size=10)
    
    def crop(self, msg):

        #Pulling image from ROS msg to cv format
        cv_img = self.bridge.imgmsg_to_cv2(msg, "bgr8")
        #Cropping image from [startY:endY, startX, endX]
        cropped_img = cv_img[100:250, 100:250]
        #Converting image from cv format to ROS msg
        ros_cropped = self.bridge.cv2_to_imgmsg(cropped_img, "bgr8")
        #publishing cropped image
        self.pub.publish(ros_cropped)



if __name__=="__main__":
    # initialize our node and create a publisher as normal
    rospy.init_node("image_cropper", anonymous=True)
    img_crop = ImageCropper()
    rospy.spin()

Similarly, you can **test the `flip_image.py` Node in the `img_proc_aux` package**

### Viewing the Image 

- Open three LXTerminals
- Build and source the catkin workspace
	- roscore 
	- rosrun img_proc_aux image_pub_all.py
	- rosrun img_proc_aux crop.py
- To view the image
	- In the VNC container, open the rqt_image_view icon.
	- In the Devel container, attach a new terminal `dts devel run attach` and run the command `rqt_image_view`


<h6><p style="text-align: center;">
<strong>Disclaimer:</strong>
This tutorial is an adaptation of the Prof. Robinette's EECE5560 UMass Lowell class.
</p></h6>